问题一：数据缺失导致可供分析的股票数减少
根据逐个数据提取的结果，绝大部分数据缺失的公司数在200个以下（小于5%），但每日指标pe_ttm缺失的公司数将近1000。如果用pe代替，缺失公司数仍在600以上。利息倍数ebit_to_interest缺失的公司数超过1000，若用ebit和interest expense代替，则几乎没有数据缺失，但会增加半小时到一小时的运行时间。以下为数据提取代码：

In [1]:
import time
import tushare as ts
import datetime
from datetime import date
from datetime import timedelta 
import numpy as np
import pandas as pd
from scipy import stats
import numpy_financial as npf
import pandas_datareader.data as web

ts.set_token('41cbc98ddd6c55a141cf281d0636c4ef25c13ff69e38bc400f3f4ebd')
pro = ts.pro_api()

/Users/miaoyuesun/anaconda3/envs/py36/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
# pe_ttm
df_daily = pro.daily_basic(ts_code=tscode, trade_date=end_date, fields='ts_code,trade_date,pe_ttm') # len = 3153
# pe
df_daily = pro.daily_basic(ts_code=tscode, trade_date=end_date, fields='ts_code,trade_date,pe') # len = 3360
# ebit_to_interest
df_indicator = pro.fina_indicator(ts_code=tscode, period=end_date, fields='ts_code,end_date,ebit_to_interest') # len = 2669
# ebit and interest expense
df_income = pro.income(ts_code=tscode, period=end_date, fields='ts_code,end_date,ebit,int_exp') # len = 4023

NameError: name 'tscode' is not defined

同时从各个数据缺失情况来看，报表中的原始数据通常比计算得出的指标数据更完整

问题二：部分数据与合理值相差较大
自由现金流free_cashflow：以茅台为例，根据大部分网站提供的数据，茅台自由现金流应在300亿到400亿，与Tushare数据相差较远。

In [23]:
pro.cashflow(ts_code='600519.SH', fields='ts_code,end_date,free_cashflow')

,ts_code,end_date,free_cashflow
0,600519.SH,20200630,3.859139e+10
1,600519.SH,20200331,5.761472e+09
2,600519.SH,20191231,-7.905213e+10
3,600519.SH,20190930,1.835904e+10
4,600519.SH,20190630,2.257577e+10
...,...,...,...
71,600519.SH,20021231,-6.203647e+07
72,600519.SH,20020630,-1.021527e+08
73,600519.SH,20011231,-1.675967e+09
74,600519.SH,20010630,NaN


问题三：部分数据名称错误
例如，interst_income实际为利息费用（利息收入-利息支出）：
df_indicator = pro.fina_indicator(ts_code=tscode, period=end_date, fields='ts_code,end_date,interst_income')

疑问：每日指标数据在从list转换为dataframe时（dropna()之前）已经缺失了超过100支股票：
daily_data_df = pd.concat(daily_data) // len(daily_data) = 4025, len(daily_data_df) = 3828
推测可能这些股票不在日线数据库，所以数据提取时存入list的是NoneType，而不仅仅是部分数据缺失。

反馈：1.针对分析结果对原来的模型进行了修改，正在最后一遍运行，明天发给您
     2.已经通过tushare QQ群问了相关数据问题，正在等待群主回复